# Traveling Wave Cavity

A traveling wave needs to be described by four fieldmaps:
entrance
two body 
exit



http://accelconf.web.cern.ch/accelconf/p79/PDF/PAC1979_3701.PDF

```
0.052464 0 0 105 1.42 25500000.0 2856000000.0 119.0 4 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /!name:entrance
2.937928 0 0 105 1.472464 29430178.7820912 2856000000.0 149.0 5 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /!name:body_1
2.937928 0 0 105 1.472464 29430178.7820912 2856000000.0 209.0 6 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /!name:body_2
0.05246 0 0 105 4.410392 25500000.0 2856000000.0 119.0 7 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /!name:exit
```


The following 4 lines define a 3-cell s-band traveling wave structure
using the supperposition of two standing wave strutures.
G. A. Loew et al., SLAC-PUB-2295, 1979.

- the phase of line 2 is the phase of line 1 + 30 degrees;
- the phase of line 3 is the phase of line 1 + 90 degrees.
- the phase of line 4 is the same as the line 1;
- the field scale of line 2 is the scale of the line 1/sin(beta d) 
- the field scale of line 3 is the scale of the line 1/sin(beta d) 
- the scale of line 4 is the same as the line 1;



In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

import numpy as np
import os

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
frf = 2856000000.0
c = 299792458.0
pi = np.pi
k = 2 * pi * frf / c
d = 3.5e-2  # periodic length
(
    np.sin(k * d),
    25905612.0 / 29913224.7,
)  # = 0.86571945106805 #roughly equals 25905612.0/29913224.7 as above

In [ ]:
frf / c

In [ ]:
from impact import Impact, fieldmaps

In [ ]:
ifile = "../templates/traveling_wave_cavity/ImpactT.in"
os.path.exists(ifile)

In [ ]:
I = Impact(ifile)

In [ ]:
# I.run()

# Fieldmaps

In [ ]:
I.input["fieldmaps"]
fmap4 = I.input["fieldmaps"]["rfdata4"]["field"]["Ez"]
fmap5 = I.input["fieldmaps"]["rfdata5"]["field"]["Ez"]
fmap6 = I.input["fieldmaps"]["rfdata6"]["field"]["Ez"]
fmap7 = I.input["fieldmaps"]["rfdata7"]["field"]["Ez"]
# fmap4 = fieldmaps.process_fieldmap_solrf(rdfa['field']['Ez']ta4['data'])['Ez']
# fmap5 = fieldmaps.process_fieldmap_solrf(rdfata5['data'])['Ez']
# fmap6 = fieldmaps.process_fieldmap_solrf(rdfata6['data'])['Ez']
# fmap7 = fieldmaps.process_fieldmap_solrf(rdfata7['data'])['Ez']

In [ ]:
fieldmaps.fieldmap_reconstruction_solrf(fmap4, 0)

In [ ]:
fmap4

In [ ]:
fmap = fmap6
zlist = np.linspace(fmap["z0"], fmap["z1"], 1000)
fieldlist = [fieldmaps.fieldmap_reconstruction_solrf(fmap, z) for z in zlist]
fig, ax = plt.subplots(figsize=(20, 5))
ax.plot(zlist, fieldlist)

In [ ]:
fieldlist5 = np.array(
    [fieldmaps.fieldmap_reconstruction_solrf(fmap5, z) for z in zlist]
)
fieldlist6 = np.array(
    [fieldmaps.fieldmap_reconstruction_solrf(fmap6, z) for z in zlist]
)

In [ ]:
l = c / frf
l

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))

wt = 2 * np.pi / 360 * 0
plt.ylim(-1, 1)
plt.xlim(0, 0.5)
ax.plot(zlist + l * 2 / 3, fieldlist5 * np.cos(wt))
ax.plot(zlist, fieldlist6 * np.cos(wt))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))

wt = 2 * np.pi / 360 * 90
plt.ylim(-1, 1)
ax.plot(zlist, fieldlist5 * np.cos(wt) + fieldlist6 * np.cos(wt + 2 * np.pi * 60 / 360))

In [ ]:
0.0586460 + 0.3371281 + 1.1518479 + 1.1515630 + 0.3351400 + 0.0609190

In [ ]:
0.0586460 + 0.3371281 + 1.1518479

In [ ]:
1.1515630 + 0.3351400 + 0.0609190

# ControlGroups

In [ ]:
from impact import ControlGroup

In [ ]:
# Add a ControlGroup that can change the relative phase

I2 = I.copy()

CAV = ControlGroup(
    ele_names=["solrf_entrance", "solrf_body_1", "solrf_body_2", "solrf_exit"],
    var_name="dtheta0_deg",
    attributes="theta0_deg",
)
CAV.link(I2.ele)
[ele["theta0_deg"] for ele in CAV.eles]

In [ ]:
CAV["dtheta0_deg"] = 0
[ele["theta0_deg"] for ele in CAV.eles]

In [ ]:
CAV_scale = ControlGroup(
    ele_names=["solrf_entrance", "solrf_body_1", "solrf_body_2", "solrf_exit"],
    var_name="rf_field_scale",
    factors=[0.86571945106805, 1, 1, 0.86571945106805],  # sin(k*d) with d = 3.5e-2 m
    absolute=True,
)
CAV_scale.link(I2.ele)

In [ ]:
CAV_scale["rf_field_scale"] = 29e6
[ele["rf_field_scale"] for ele in CAV_scale.eles]

# Autophase and scale

In [ ]:
#  Changes in phases
I.add_group(
    "L0A",
    ele_names=["solrf_entrance", "solrf_body_1", "solrf_body_2", "solrf_exit"],
    var_name="theta0_deg",
    attributes="theta0_deg",
)


# Overall scaling, respecting the special factors.
I.add_group(
    "L0A_scale",
    ele_names=["solrf_entrance", "solrf_body_1", "solrf_body_2", "solrf_exit"],
    var_name="rf_field_scale",
    factors=[0.86571945106805, 1, 1, 0.86571945106805],  # sin(k*d) with d = 3.5e-2 m
    absolute=True,
)

In [ ]:
from impact.autophase import autophase_and_scale

from pmd_beamphysics import single_particle

# Start particles at 1.4 m, just in front of the cavity
P0 = single_particle(pz=6e6, z=1.4)

In [ ]:
autophase_and_scale(
    I,
    phase_ele_name="L0A",
    scale_ele_name="L0A_scale",
    target=64e6,
    scale_range=(10e6, 100e6),
    initial_particles=P0,
    verbose=True,
)

In [ ]:
PF = I.track(P0, 4.5)
PF["mean_energy"]

# Track distgen particles

In [ ]:
from distgen import Generator

YAML = """
n_particle: 20000
random_type: hammersley
species: electron
start:
  tstart:
    units: sec
    value: 0
  type: time
total_charge:
  units: pC
  value: 250.0
r_dist:
  sigma_xy:
    units: mm
    value: 0.01
  type: radial_gaussian
z_dist:
  avg_z:
    units: m
    value: 1.4
  sigma_z:
    units: mm
    value: 0.01
  type: gaussian
  

transforms:
  setPz:
    type: set_avg pz
    avg_pz: 
      value: 6
      units: MeV/c
  
"""
G = Generator(YAML)
G.run()
DP = G.particles
DP.plot("z", "pz")

In [ ]:
DP["min_z"]

In [ ]:
I.header["Bcurr"] = 0  # SC off
I.header["Dt"] = 1e-11
PF = I.track(DP, 5)

In [ ]:
PI = I.particles["initial_particles"]
PI.plot("z", "x")

In [ ]:
PF.plot("delta_z", "delta_pz")

In [ ]:
# Compare these.
key1 = "mean_z"
key2 = "mean_kinetic_energy"
units1 = str(I.units(key1))
units2 = str(I.units(key2))
plt.xlabel(key1 + f" ({units1})")
plt.ylabel(key2 + f" ({units2})")
plt.plot(I.stat(key1), I.stat(key2))
plt.scatter(
    [I.particles[name][key1] for name in I.particles],
    [I.particles[name][key2] for name in I.particles],
    color="red",
)